# Analyzing CIA Factbook Data Using SQL

This project aims at analyzing demographical and geographical characteristics of countries in the world based on data in the following database:
[CIA Factbook Data](https://www.cia.gov/the-world-factbook/).

# Connect to the database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

# Overview of data

We are going to have an overview of the database.

The following are the descriptions for some of the columns given:

- `name` — the name of the country.
- `area` — the country's total area (both land and water).
- `area_land` — the country's land area in square kilometers.
- `area_water` — the country's waterarea in square kilometers.
- `population` — the country's population.
- `population_growth` — the country's population growth as a percentage.
- `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

In [6]:
%%sql
SELECT *
FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Population

Let's start by looking at the maximum and minimum values of population and population growth of the countries.

In [11]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


There is a country that does not contain any inhabitants, as shown by the minimum value of population, 0; Meanwhile, there is a country that contains 7.2 billion of inhabitants!

Let's explore the data of these two countries.

# Exploring outliers

__Country that does not contain any inhabitant:__

In [17]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MIN(population)
                           FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


__Country that contain max population:__

In [18]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MAX(population)
                    FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


__Minimum population:__ The country that does not contain inhabitant is Antarctica. By looking into the [page for Antarctica](https://www.cia.gov/the-world-factbook/countries/antarctica/), we found that Antarctica was actually a protected natural reserve with no indigenous inhabitants, but both year-round and summer-only staffed research stations.

__Maximum population:__ The country is actually 'World', which is the consolidation of data of all countries. We can keep in mind to exclude this row in our analysis.

# Excluding world data

Let's re-explore the data by excluding `World` row.

In [19]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
FROM facts
WHERE name <> 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


The maximum population is now 1367 million.

# Densely populated countries

In [24]:
%%sql
SELECT AVG(population) AS avg_pop,
       AVG(area) AS avg_area
FROM facts
WHERE name <> 'World';

Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


In [70]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts
                   WHERE name <> 'World')
AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


The are 14 countries as listed above that contain:
- Higher than average population AND
- Lower than average area

These 14 countries are:
- Bangladesh
- Germany
- Iraq
- Italy
- Japan
- South Korea
- Morocco
- Philippines
- Poland
- Spain
- Thailand
- Uganda
- United Kingdom
- Vietnam

# Exploring other population characteristics

__Country with the highest population:__

In [40]:
%%sql
SELECT name, population
FROM facts
WHERE population == (SELECT MAX(population)
                    FROM facts
                    WHERE name <> 'World');

Done.


name,population
China,1367485388


__Country with the highest population growth:__

In [41]:
%%sql
SELECT name, population, population_growth
FROM facts
WHERE population_growth == (SELECT MAX(population_growth)
                           FROM facts
                           WHERE name <> 'World');

Done.


name,population,population_growth
South Sudan,12042910,4.02


__Country with the most number of people in the coming year:__

In [68]:
%%sql
SELECT name, MAX((population)*(1+population_growth/100)) AS pop_next_year
FROM facts
WHERE name <> 'World';

Done.


name,pop_next_year
China,1373639072.2459998


__Countries where death rate is greater than birth rate:__

In [47]:
%%sql
SELECT name AS country, birth_rate, death_rate
FROM facts
WHERE (birth_rate < death_rate);

Done.


country,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


__Countries with the highest density:__

In [59]:
%%sql
SELECT name AS country, population/area AS density
FROM facts
ORDER BY density DESC
LIMIT 10;

Done.


country,density
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299


__Observation:__

There are some other observations from the analysis:
1. China is the country with the world's highest population, with 1367 million of inhabitants. It will continue to be the country with the highest population in the coming year.
2. South Sudan has the highest population growth rate among the world, of 4.02%
3. There are 24 countries with high death rate than birth rate, 21 of which are European countries. The remaining 3 countries are Japan, Russia and Saint Pierre and Miquelon
4. The most densed countries by calculating the population to area ratio are:
    - Macau
    - Monaco
    - Singapore
    - Hong Kong
    - Gaza Strip
    - Gibraltar
    - Bahrain
    - Maldives
    - Malta
    - Bermuda

Interestingly, the most densed countries by calculating population to area ratio are different compared to how we look at them earlier. This method gives a more accurate result comparatively.

# Exploring water to land ratio

In [63]:
%%sql
SELECT name AS country, 
    CAST(area_water AS Float)/area_land AS 'water_to_land_ratio'
FROM facts
ORDER BY water_to_land_ratio DESC
LIMIT 10;

Done.


country,water_to_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


In [64]:
%%sql
SELECT name AS country, area_water, area_land
FROM facts
WHERE area_water > area_land;

Done.


country,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


From the analysis, we found that there are two countries that have much bigger water area than land:
- British Indian Ocean Territory
- Virgin Islands